# DBMS Innovative Project
# <u>Artififial Neural Network implementation on UCI-dataset</u>
### Submitted by: <br>Anubhav Maheshwari 2K19/EE/048 <br>Anuj Majumdar 2K19/EE/050
---
#### A 2-hidden-layer artificial neural network has been implemented on Breast Cancer Dataset from the UCI Machine Learning Repository
Link to dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/
<br> <br>The implementation is done from scratch and no external machine learning libraries were used. The neural network is trained using different activation functions such as 
<ul>
    <li>sigmoid $ y = \frac{1}{1+e^{-x}} $
    <li>tanh  $ y = \frac{e^{x}-e^{-x}}{e^{x}+e^{-x}} $
    <li>ReLU $ y = max(0, x) $
</ul>
    Training and testing error sum for different activation functions is calculated and compared to get the best activation function on the given dataset.

---

##### Importing libraries: 
<ol>
    <li><u>NUMPY</u> for working with arrays
    <li><u>PANDAS</u> for datascience application
    <li><u>SKLEARN</u> for modeling functions like filling missing values, standardize the format, splitting data into training and testing
</ol>

In [15]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

---
### Variables used:
<ul>
    <li>h1 - number of neurons in the first hidden layer
    <li>h2 - number of neurons in the second hidden layer
    <li>X - vector of features for each instance
    <li>y - output for each instance
    <li>w01, delta01, X01 - weights, updates and outputs for connection from layer 0 (input) to layer 1 (first hidden)
    <li>w12, delata12, X12 - weights, updates and outputs for connection from layer 1 (first hidden) to layer 2 (second hidden)
    <li>w23, delta23, X23 - weights, updates and outputs for connection from layer 2 (second hidden) to layer 3 (output layer)</ul>

---

##### Creating the Artificial Neural Network
The neural network class was created with the following functions:
<ol>
    <li>__init__: Constructor function for train_test_split. The raw data was first read from the csv file using pandas. Data preprocessing was done. Number of input and output layers were found. The data was split into training and testing. Random weights were assigned to the nodes in the network.
    <li> __sigmoid:  $ y = \frac{1}{1+e^{-x}} $
    <li> __sigmoid_derivative: $ y = \frac{e^{-x}}{(1+e^{-x})^2} $
    <li> __tanh: $ y = \frac{e^{x}-e^{-x}}{e^{x}+e^{-x}} $
    <li> __tanh_derivative: $ y = 1-tanh^2(x) $
    <li> __relu: y = max(0, x)
    <li> __relu_derivative: $y = 0; $ if $x \leq 0$<br>$y = 1; $ if $x>0$
    <li> __activation: This is a menu driven function to select the activation function to be used.
    <li> __activation_derivative: This is a menu driven function to select the derivatives of activation function to be used. 
    <li> preprocess: Standardization, normalization, handling null values and converting categorical data to numerical of the dataset.
    <li> compute_hidden_layer1_delta: Application of delta rule to first hidden layer
    <li> compute_hidden_layer2_delta: Application of delta rule to second hidden layer
    <li> compute_output_delta: Application of delta rule to output layer
    <li> train: Function to train the data
    <li> forward_pass: Passing our inputs through the neural network towards output
    <li> backward_pass: Backpropagation through the neural network using the delta rule
    <li> predict: Assuming that the test dataset has the same format as the training dataset, error in output is predicted.
</ol>

In [13]:
class NeuralNet:
    def __init__(self, train, train_test_split_size, header = True, h1 = 4, h2 = 2):
        
        #generating pseudo-random numbers
        np.random.seed(1)
        
        # train refers to the training dataset
        # test refers to the testing dataset
        # h1 and h2 represent the number of nodes in 1st and 2nd hidden layers

        raw_input = pd.read_csv(train, header=None)
        
        # Preprocessing
        train_dataset = self.preprocess(raw_input)
        ncols = len(train_dataset.columns)
        nrows = len(train_dataset.index)
        self.X = train_dataset.iloc[:, 0:(ncols -1)].values.reshape(nrows, ncols-1)
        self.y = train_dataset.iloc[:, (ncols-1)].values.reshape(nrows, 1)
        
        # Finding number of input and output layers from the dataset
        input_layer_size = len(self.X[0])
        if not isinstance(self.y[0], np.ndarray):
            output_layer_size = 1
        else:
            output_layer_size = len(self.y[0])

        # Split dataset into training set and testing set on the basis of test set size
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=train_test_split_size)

        # assign random weights to matrices in network
        # number of weights connecting layers = (no. of nodes in previous layer) x (no. of nodes in following layer)
        self.w01 = 2 * np.random.random((input_layer_size, h1)) - 1
        self.X01 = self.X_train
        self.delta01 = np.zeros((input_layer_size, h1))
        self.w12 = 2 * np.random.random((h1, h2)) - 1
        self.X12 = np.zeros((len(self.X_train), h1))
        self.delta12 = np.zeros((h1, h2))
        self.w23 = 2 * np.random.random((h2, output_layer_size)) - 1
        self.X23 = np.zeros((len(self.X_train), h2))
        self.delta23 = np.zeros((h2, output_layer_size))
        self.deltaOut = np.zeros((output_layer_size, 1))
    
    # sigmoid function
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # derivative of sigmoid function, indicates confidence about existing weight

    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # tanh function
    def __tanh(self, x):
        return np.tanh(x)

    # derivative of tanh function
    def __tanh_derivative(self, x):
        return 1 - np.tanh(x) ** 2

    # ReLu function
    def __relu(self, x):
        return np.maximum(0, x)

    # derivative of Relu function (Assuming a derivative value of 0 for x=0)
    def __relu_derivative(self, x):
        return (x > 0) * 1

    
    # Menu driven function to select the activation function
    
    def __activation(self, x, activation="sigmoid"):
        if activation == "sigmoid":
            return self.__sigmoid(x)
        elif activation == "tanh":
            return self.__tanh(x)
        elif activation == "relu":
            return self.__relu(x)
        return None

    
    # Menu driven function to select the derivatives of activation functions
    
    def __activation_derivative(self, x, activation="sigmoid"):
        if activation == "sigmoid":
            self.__sigmoid_derivative(x)
        elif activation == "tanh":
            self.__tanh_derivative(x)
        elif activation == "relu":
            self.__relu_derivative(x)
    
    
    
    # Pre-processing would include standardization, normalization, categorical to numerical, etc
    def preprocess(self, X):
        df = X

        #Convert categorical attributes to numerical attributes
        for col in df:
            if df[col].dtype == 'object':
                df[col] = df[col].astype('category').cat.codes.astype('int64')

        arr = df.values

        #Handle null or missing values
        imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
        imputer = imputer.fit(arr)
        arr = imputer.transform(arr)

        #Standardization, converting mean to 0 and standard deviation to 1
        scaler = StandardScaler().fit(arr)
        arr = scaler.transform(arr)

        df = pd.DataFrame(arr)
        return df
    
    # Application of delta rule to first hidden layer
    def compute_hidden_layer1_delta(self, activation="sigmoid"):
        prod = self.delta23.dot(self.w12.T)
        delta_hidden_layer1 = None
        if activation == "sigmoid":
            delta_hidden_layer1 = prod * (self.__sigmoid_derivative(self.X12))
        elif activation == "tanh":
            delta_hidden_layer1 = prod * (self.__tanh_derivative(self.X12))
        elif activation == "relu":
            delta_hidden_layer1 = prod * (self.__relu_derivative(self.X12))
        self.delta12 = delta_hidden_layer1
        
    # Application of delta rule to second hidden layer
    def compute_hidden_layer2_delta(self, activation="sigmoid"):
        prod = self.deltaOut.dot(self.w23.T)
        delta_hidden_layer2 = None
        if activation == "sigmoid":
            delta_hidden_layer2 = prod * (self.__sigmoid_derivative(self.X23))
        elif activation == "tanh":
            delta_hidden_layer2 = prod * (self.__tanh_derivative(self.X23))
        elif activation == "relu":
            delta_hidden_layer2 = prod * (self.__relu_derivative(self.X23))
        self.delta23 = delta_hidden_layer2
    
    # Application of delta rule to output layer
    def compute_output_delta(self, out, activation="sigmoid"):
        diff = self.y_train - out
        delta_output = None
        if activation == "sigmoid":
            delta_output = diff * (self.__sigmoid_derivative(out))
        elif activation == "tanh":
            delta_output = diff * (self.__tanh_derivative(out))
        elif activation == "relu":
            delta_output = diff * (self.__relu_derivative(out))
        self.deltaOut = delta_output
    
    # Passing our inputs through the neural network towards output
    def forward_pass(self, input, activation="sigmoid"):
        in1 = np.dot(input, self.w01)
        self.X12 = self.__activation(in1, activation)
        in2 = np.dot(self.X12, self.w12)
        self.X23 = self.__activation(in2, activation)
        in3 = np.dot(self.X23, self.w23)
        out = self.__activation(in3, activation)
        return out
    # Backpropagation through the neural network using the delta rule
    def backward_pass(self, out, activation="sigmoid"):
        self.compute_output_delta(out, activation)
        self.compute_hidden_layer2_delta(activation)
        self.compute_hidden_layer1_delta(activation)

    # Training function
    def train(self, activation="sigmoid", max_iterations = 1000, learning_rate = 0.05):
        for iteration in range(max_iterations):
            out = self.forward_pass(self.X_train, activation)
            error = 0.5 * np.power((out - self.y_train), 2)
            self.backward_pass(out, activation)
            update_layer2 = learning_rate * self.X23.T.dot(self.deltaOut)
            update_layer1 = learning_rate * self.X12.T.dot(self.delta23)
            update_input = learning_rate * self.X01.T.dot(self.delta12)
            self.w23 = self.w23 + update_layer2
            self.w12 = self.w12 + update_layer1
            self.w01 = self.w01 + update_input
        print("After " + str(max_iterations) + " iterations, and having learning rate as " + str(learning_rate) + ", the total error is " + str(np.sum(error)))
        print("The final weight vectors are (starting from input to output layers)")
        print(self.w01)
        print(self.w12)
        print(self.w23)

    # Implementing the predict function for applying the trained model on the  test dataset.
    # assuming that the test dataset has the same format as the training dataset
    def predict(self, activation="sigmoid", header = True):
        out = self.forward_pass(self.X_test, activation)
        error = 0.5 * np.power((out - self.y_test), 2)
        return np.sum(error)


---
## Main function
We select the activation function to be used to train the ANN.
<br>10% of the data is the testing data and 90% is the training data.
<br>Maximum iterations are set at 2000, with 5% learning rate.
<br>Training is done and finally error sum percentage is given as output.

---

In [19]:
if __name__ == "__main__":
    inp = input("Press the following keys for the activation functions: \n Press 1 for Sigmoid \n Press 2 for Tanh \n Press 3 for ReLu \n Pressing any other key will result in the activation function being sigmoid")
    if inp=="2":
        activation = "tanh"
    elif inp=="3":
        activation = "relu"
    else:
        activation = "sigmoid"

    #Specifying the train_test split. The value of train_test_split_size indicates that much % of testing data and remaining % of training data
    train_test_split_size = 0.10

    #Specifying the maximum number of iterations to train the neural network
    max_iteratons = 2000

    #Specifying the learning rate
    learning_rate = 0.05

    #Specify the dataset csv file (Using Breast cancer dataset from UCI by default)
    #Breast Cancer UCI data link: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data
    dataset_file="https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data"

    print("Training on " + str((1-train_test_split_size)*100) + "% data and testing on " + str(train_test_split_size*100) + "% data using the activation function as " + str(activation))

    #Initialize the neural network
    neural_network = NeuralNet(dataset_file, train_test_split_size)

    #Train the neural network
    neural_network.train(activation, max_iteratons, learning_rate)

    #Test the neural network
    testError = neural_network.predict(activation)
    print("Testing error sum using activation function as " + str(activation) + ": " + str(testError))

Press the following keys for the activation functions: 
 Press 1 for Sigmoid 
 Press 2 for Tanh 
 Press 3 for ReLu 
 Pressing any other key will result in the activation function being sigmoid3
Training on 90.0% data and testing on 10.0% data using the activation function as relu
After 2000 iterations, and having learning rate as 0.05, the total error is 129.79519697787373
The final weight vectors are (starting from input to output layers)
[[ 0.61667749 -0.26324629  0.21841398 -0.93030455]
 [-0.29084554 -0.84296007  0.3863704  -0.97457467]
 [-0.08090943  0.92263452 -0.33162956 -0.05583316]
 [-0.7892175   0.00615181  0.77137972  0.06875468]
 [-0.43704647 -0.29083062  0.7925605  -0.51702172]
 [-0.95223181  0.93145361 -0.1400642  -0.30742297]
 [ 0.15413526 -0.74694768  0.90008662 -0.37277834]
 [ 0.90566225 -0.56321354 -0.5025996   0.72760359]
 [-0.52946334  0.63020011  0.08779447 -0.61747946]]
[[ 0.17767988 -0.90372641]
 [-0.96825404 -0.90057797]
 [-0.20116942  0.15380792]
 [ 0.73503491  